In [1]:
import google_play_scraper

In [2]:
from google_play_scraper import app

result = app(
    'com.binance.dev',
    lang='en', # defaults to 'en'
    country='us' # defaults to 'us'
)

In [3]:
print('App detail : \n' , result)

App detail : 
 {'title': 'Binance: Buy Bitcoin & Crypto', 'description': 'Securely buy, sell, and hold cryptocurrencies like Bitcoin (BTC), Ethereum (ETH), Solana (SOL), Notcoin (NOT) and PEPE (PEPE) with low trading fees. Used by over 275 million users worldwide, Binance is the largest cryptocurrency exchange* in the world.\r\nHere is why:\r\n\r\nTRADE YOUR FAVORITE TOKENS AND MORE\r\n\r\nTrade over 350 listed cryptocurrencies, including Bitcoin (BTC), Ethereum (ETH), PEPE (PEPE) and Notcoin (NOT).\r\nTrack the market using price alerts and trade with advanced trading tools. \r\nSet up recurring orders (DCA) to buy crypto every hour, day, week or month.\r\nEnjoy best-in-class liquidity on each crypto trade. \r\nBuy cryptocurrency and fund your wallet in minutes with flexible payment options, including credit/debit, bank transfers, and peer-to-peer (P2P) trading.\r\nFind leading traders and replicate their trading strategies with one tap.\r\n\r\nEARN DAILY REWARDS ON YOUR IDLE ASSETS\r

In [4]:
import pandas as pd
import os
from google_play_scraper import Sort, reviews_all, reviews
result, continuation_token = reviews(
    'com.binance.dev',
    lang='en', # defaults to 'en'
    country='us', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.NEWEST
    count=3, # defaults to 100
    filter_score_with=3 # defaults to None(means all score)
)

# If you pass `continuation_token` as an argument to the reviews function at this point,
# it will crawl the items after 3 review items.

result, _ = reviews(
    'com.binance.dev',
    continuation_token=continuation_token # defaults to None(load from the beginning)
)

print('Results of 3 latest reviews, language English, country US, review score is 3 : \n', result)

Results of 3 latest reviews, language English, country US, review score is 3 : 
 [{'reviewId': 'c9c7e958-a270-4596-89df-acdb3ec5df92', 'userName': 'Ahsan Khan', 'userImage': 'https://play-lh.googleusercontent.com/a-/ALV-UjVABN7VPbtqonGvXCB--DQisVkvJOrvgEZ-7lDohrGuF4miU_9G', 'content': 'Nice', 'score': 3, 'thumbsUpCount': 0, 'reviewCreatedVersion': '3.3.3', 'at': datetime.datetime(2025, 9, 25, 10, 59, 54), 'replyContent': 'Hi there! We are pleased that we met your expectations. Would you tell us what is missing for a 5-star rating? So, we can improve ourselves according to your feedback. 🌟 That would be very encouraging for us. Stay SAFU and have a nice day. Thanks 😊 ^WI\n', 'repliedAt': datetime.datetime(2025, 9, 25, 14, 49, 36), 'appVersion': '3.3.3'}, {'reviewId': '2ee39d89-cbf6-44fb-aee7-41cb44a1619d', 'userName': 'Akbar Ali', 'userImage': 'https://play-lh.googleusercontent.com/a/ACg8ocKjE75Dau481oaja7lLui9DOgK4fU-eFPpd4VeTKsFnv_FArw=mo', 'content': 'good', 'score': 3, 'thumbsUpCoun

In [ ]:
"""
Collect last-5-years English reviews from Google Play for Zerodha, Groww, Coinbase, Binance.
Saves a CSV per app in the current working directory.

Requires:
    pip install google-play-scraper pandas

Note:
 - This script uses google_play_scraper.reviews_all to fetch reviews (lang='en').
 - It filters reviews to keep only those with timestamp within the last 5 years.
 - It saves cleaned CSVs (ISO datetime strings) to the current working directory.
"""

import os
import datetime
import pandas as pd
from google_play_scraper import reviews_all, Sort

# app_id dictionary: application name -> Google Play package name
app_id = {
    "zerodha_kite": "com.zerodha.kite3",           # Play store listing for Kite (Zerodha)
    "groww": "com.nextbillion.groww",             # Groww Android package (Play Store)
    "coinbase": "com.coinbase.android",           # Coinbase Android package (Play Store)
    "binance": "com.binance.dev",                 # Binance global Android package (Play Store)
}
# Sources for package names (Google Play pages): :contentReference[oaicite:0]{index=0}

# how far back to collect (approximate 5 years)
DAYS_5_YEARS = 365 * 5
cutoff_dt = datetime.datetime.utcnow() - datetime.timedelta(days=DAYS_5_YEARS)

# fields to remove before saving (explainable, optional)
FIELDS_TO_REMOVE = {
    "userImage",
    "reviewCreatedVersion",
    "replyContent",
    "repliedAt",
    # keep 'at' for filtering, but will be converted to ISO string before saving
}

# polite delay between internal requests in reviews_all (milliseconds)
SLEEP_MS = 0  # set >0 (e.g., 100) if you hit rate limits

def fetch_reviews_for_app(pkg_name: str, lang: str = "en"):
    """
    Fetch all reviews (via reviews_all) for the given package name & language.
    Returns list of review dicts (raw).
    """
    # reviews_all returns a list of dicts with keys like:
    # 'content','score','at','reviewCreatedVersion','userName','userImage','replyContent','repliedAt','reviewId'
    reviews = reviews_all(
        pkg_name,
        sleep_milliseconds=SLEEP_MS,
        lang=lang,
        sort=Sort.NEWEST,
        count=None  # fetch all available (we'll filter by date)
    )
    return reviews

def filter_reviews_last_n_years(reviews: list, cutoff_datetime: datetime.datetime):
    """
    Keep only reviews with review['at'] >= cutoff_datetime.
    reviews: list of dicts where 'at' is a datetime.datetime object.
    """
    filtered = [r for r in reviews if ("at" in r and isinstance(r["at"], datetime.datetime) and r["at"] >= cutoff_datetime)]
    return filtered

def clean_review_dicts(reviews: list):
    """
    Remove unneeded keys and convert datetime to ISO strings.
    """
    cleaned = []
    for r in reviews:
        rr = r.copy()
        # convert datetime to ISO string if present
        if "at" in rr and isinstance(rr["at"], datetime.datetime):
            rr["at"] = rr["at"].astimezone(datetime.timezone.utc).isoformat()
        # remove fields we don't want in final CSV
        for k in list(FIELDS_TO_REMOVE):
            if k in rr:
                del rr[k]
        cleaned.append(rr)
    return cleaned

def save_reviews_csv(reviews: list, filename: str):
    """
    Save list-of-dicts to CSV via pandas.
    """
    if not reviews:
        print(f"[info] No reviews to save for {filename}. Skipping file write.")
        return
    df = pd.DataFrame(reviews)
    # Ensure deterministic column order: put 'content','score','at','reviewId' first if present
    cols = list(df.columns)
    preferred = [c for c in ["reviewId", "content","score","at","userName"] if c in cols]
    other = [c for c in cols if c not in preferred]
    df = df[preferred + other]
    df.to_csv(filename, index=False)
    print(f"[saved] {len(df)} rows -> {filename}")

def main():
    for app_name, pkg in app_id.items():
        print(f"\n=== Fetching reviews for {app_name} (pkg: {pkg}) ===")
        try:
            raw_reviews = fetch_reviews_for_app(pkg, lang="en")
        except Exception as e:
            print(f"[error] Failed to fetch reviews for {app_name} ({pkg}): {e}")
            continue

        print(f"[fetched] {len(raw_reviews)} total reviews fetched (lang=en) for {app_name}.")

        # Filter to last 5 years
        recent = filter_reviews_last_n_years(raw_reviews, cutoff_dt)
        print(f"[filtered] {len(recent)} reviews from last ~5 years (cutoff {cutoff_dt.date()}).")

        # Clean and save
        cleaned = clean_review_dicts(recent)
        out_fname = f"{app_name}_reviews_last5y.csv"
        save_reviews_csv(cleaned, out_fname)

if __name__ == "__main__":
    main()


C:\Users\avane\AppData\Local\Temp\ipykernel_24448\65873665.py:30: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  cutoff_dt = datetime.datetime.utcnow() - datetime.timedelta(days=DAYS_5_YEARS)



=== Fetching reviews for zerodha_kite (pkg: com.zerodha.kite3) ===
[fetched] 119818 total reviews fetched (lang=en) for zerodha_kite.
[filtered] 80233 reviews from last ~5 years (cutoff 2020-09-28).
[saved] 80233 rows -> zerodha_kite_reviews_last5y.csv

=== Fetching reviews for groww (pkg: com.nextbillion.groww) ===
[fetched] 369555 total reviews fetched (lang=en) for groww.
[filtered] 322726 reviews from last ~5 years (cutoff 2020-09-28).
[saved] 322726 rows -> groww_reviews_last5y.csv

=== Fetching reviews for coinbase (pkg: com.coinbase.android) ===
[fetched] 176934 total reviews fetched (lang=en) for coinbase.
[filtered] 117726 reviews from last ~5 years (cutoff 2020-09-28).
[saved] 117726 rows -> coinbase_reviews_last5y.csv

=== Fetching reviews for binance (pkg: com.binance.dev) ===
[fetched] 332704 total reviews fetched (lang=en) for binance.
[filtered] 323457 reviews from last ~5 years (cutoff 2020-09-28).
[saved] 323457 rows -> binance_reviews_last5y.csv


: 